# Loads

In [1]:
%matplotlib widget

# This thing just saves some whitespace from a bug in tqdm / notebook
from IPython.display import HTML
display(HTML("""
<style>
.jp-OutputArea-prompt:empty {
  padding: 0;
  border: 0;
}
</style>
"""))

In [2]:
from importlib import reload
import numpy as np
import matplotlib.pyplot as plt
from genSTEM import Model, Correct

from ase.spacegroup import crystal
from ase.build import make_supercell

# Build an image

Build the structure with ase

In [3]:
a = 9.04
skutterudite = crystal(('Co', 'Sb'),
                       basis=[(0.25, 0.25, 0.25), (0.0, 0.335, 0.158)],
                       spacegroup=204,
                       cellpar=[a, a, a, 90, 90, 90])

atoms = make_supercell(skutterudite, np.diag([4,4,1]))
atoms.numbers[330] = 80
atoms.numbers[331] = 80
atoms.numbers[346] = 80

Build the image with genSTEM

In [4]:
images, scanangles, random_angle = Model.get(atoms, drift_speed=12, vacuum=10, nImages=2, pixel_size=0.2, jitter=0, maxScanAngle=180, drift_angle=0)

  0%|          | 0/2 [00:00<?, ?it/s]

Size: 0.001227664 GB
Shape: (2, 277, 277)


Here we add one line of artificcial jitter so that we can analyze the effect of an affine transformation mapped on scan imperfections.

In [5]:
images[0, 100, :272] = images[0, 100, 5:277]
images[1, 150, :272] = images[1, 150, 5:277]

Plot the generated images

In [6]:
ncols = 4 if len(images) > 4 else len(images)
mult = 12 / ncols
nrows = len(images) // ncols if len(images) % ncols == 0 else len(images) // ncols +1
rowcols = np.array([nrows, ncols])

fig, AX = plt.subplots(*rowcols, figsize=mult*rowcols[::-1])
for ax, img in zip(AX.flatten(), images):
    img = Correct.asnumpy(img)
    ax.imshow(img, cmap='jet')
    ax.axis('off')

limits = images.min(), images.max()
plt.figure()
img = Correct.create_final_stack_and_average(images, scanangles, 0, 0, print_shifts=True)
img[img == 0] = np.nan
plt.imshow(img, clim=limits, cmap='jet')
plt.title('Registration without Affine Correction')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Registration without Affine Correction')

# Drift detection and correction
We begin by finding the global sample drift by itteratively performing hybrid correlation. <br />
The following cell plots the convergance of the hybrid correlation performed in the successive cell.

In [7]:
fig = plt.figure(figsize = (12,4)) 

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
print(f"\nActual angle: {np.round(np.rad2deg(random_angle), 2)}°")
best_angle, best_speed, history = Correct.estimate_drift(images, scanangles, tolerancy_percent=2, normalize_correlation=True, debug=False)
print(f"Missed by {np.round(best_angle - np.rad2deg(random_angle),2)}°")


Actual angle: 0.0°
Getting fit with no drift first


Iterating through drift angles:   0%|          | 0/1 [00:00<?, ?it/s]

Iterating through drift speeds:   0%|          | 0/1 [00:00<?, ?it/s]


Iteration #0: Best guess: Angle: 0°, Speed: 0.00e+00. Fit value: 8.87e+06
Limits: [0.0°, 315.0°], [0.00e+00, 3.61e-03]


Iterating through drift angles:   0%|          | 0/8 [00:00<?, ?it/s]

...

Iterating through drift speeds:   0%|          | 0/8 [00:00<?, ?it/s]

Iterating through drift speeds:   0%|          | 0/8 [00:00<?, ?it/s]

Iterating through drift speeds:   0%|          | 0/8 [00:00<?, ?it/s]

Iterating through drift speeds:   0%|          | 0/8 [00:00<?, ?it/s]

Adjusting drift speed limits
Final iteration: Final guess: Angle: 1.09°, Speed: 7.89e-04

Took 20.0 seconds
Missed by 1.09°


Correct for the global drift

In [9]:
imgs = Correct.warp_and_shift_images(images, scanangles, best_speed, best_angle)

Plot the original images

In [10]:
fig, AX = plt.subplots(*rowcols, figsize=mult*rowcols[::-1])
for ax, img in zip(AX.flatten(), images):
    ax.imshow(Correct.asnumpy(img))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Plot a comparison of reconstruction after a rigid registration and registration with afine transforms with/without subpixel accuracy.

In [12]:
fig, axs = plt.subplots(ncols=3, nrows=2, figsize=(9,6))

cent = images.shape[1]//2

img1 = Correct.create_final_stack_and_average(images, scanangles, 0, 0)
fft1 = np.abs(np.fft.fftshift(np.fft.fft2(img1)))
axs[0,0].set_title('Only Registration')
axs[0,0].imshow(img1, cmap='jet')
axs[1,0].imshow(np.log10(fft1[cent-25:cent+25,cent-25:cent+25]), cmap='jet')


img2 = Correct.create_final_stack_and_average(images, scanangles, best_speed, best_angle)
fft2 = np.abs(np.fft.fftshift(np.fft.fft2(np.nan_to_num(img2))))
axs[0,1].set_title('Affine Transformation\nand Registration')
axs[0,1].imshow(img2, cmap='jet')
axs[1,1].imshow(np.log10(fft2[cent-25:cent+25,cent-25:cent+25]), cmap='jet')

img3 = Correct.create_final_stack_and_average(images, scanangles, best_speed, best_angle, subpixel=True)
fft3 = np.abs(np.fft.fftshift(np.fft.fft2(np.nan_to_num(img3))))
axs[0,2].set_title('Affine Transformation\nand Subpixel Registration')
axs[0,2].imshow(img3, cmap='jet')
axs[1,2].imshow(np.log10(fft3[cent-25:cent+25,cent-25:cent+25]), cmap='jet')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …